### Importando módulos

In [1]:
import glob, os, scipy, nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from heapq import nlargest
from sklearn.feature_extraction.text import CountVectorizer


### Funções utilizadas no processamento 

In [122]:
def preProcessingCorpus(folder,searchWords):

    #Criando o arquivo
    files = glob.glob(os.path.join(folder,'*.txt'))
    
    #Criando o nova dataframe
    df = pd.DataFrame()
    count = 0
    
    #Percorrendo todos os arquivo
    for file in files:      
    
        #Coletando informacoes do contidas no arquivo e nome do arquivo com docid
        data = open(file,'r',encoding='utf-8')
        docid = str(os.path.basename(file)[0:-4])
        year = str(docid.split('-')[2])
    
        #inserindo valores no dataframe e limpando arquivos da memoria
        datas = pd.DataFrame({'id':[count],'docid':[docid],'text':[data.read()],
                              'year':[year],'ausência':[0],'critério':[0],'decisão':[0],'inverídica':[0]
                             })
        
        datas = np.concat([pd.DataFrame({'id':[count],'docid':[docid],'text':[data.read()]}),
                           pd.DataFrame({'id':[count],'docid':[docid],'text':[data.read()]})
                          ])
        
        df = df.append(datas)
        datas = None
    
        #Fechando o arquivo
        data.close()
        count += 1
    return df

def processCorpus(dataframe,fieldtext):
    
    #Retirando espacos e colando as palavras minuscula
    wpt = WordPunctTokenizer()
    dataframe[fieldtext] = (dataframe[fieldtext].str.lower()).apply(wpt.tokenize) 
    
    #Retirando os sufixos das palavras
    stm = PorterStemmer()
    texto_final_matrix = list()
    for texto in dataframe[fieldtext]:    
        texto_final = [stm.stem(txt) for txt in texto]
        texto_final_matrix.append(texto_final)
    dataframe[fieldtext] = texto_final_matrix

    #Retirando as stopwords e as pontuacoes
    #Coletando as stopword em Português
    stopw = stopwords.words('portuguese') + list(punctuation)
        
    #Retirando as stopwords
    palavras_sem_stopwords = []
    for word_final in dataframe[fieldtext]:
        palavras_sem_stopwords.append([word for word in word_final if word not in stopw])

    dataframe[fieldtext] = palavras_sem_stopwords
     
    #Retornando o resultado da função    
    return dataframe
    
def getCountWordsDocuments(dataframe,fieldtext):
    
    #Frequencia das palavras
    rows,cols = dataframe.shape
    
    #Percorrer todo o dataframe
    for row in range(rows):    
        
        #Criando os objeto das palavras chaves
        objs = {'ausência':[0],'critério':[0],'decisão':[0],'inverídica':[0]}
        
        #it = iter(listKeys)#zeros = [0]*len(listKeys)#objs = dict(zip(it,zeros))
        
        #Set variavel dos textos de cada linha do dataframes
        rowWord = dataframe.loc[dataframe['id'] == row,fieldtext]
        words = [words for words in rowWord[0]]
        
        #Percorrer cada palavra do texto e quantifica as palavras chaves
        for word in words:
            if word in objs.keys():
                objs[word] = [objs[word][0] + 1]
        
        #Atualizando o dataframe
        dataframe.loc[dataframe['id'] == row,list(objs.keys())] = pd.DataFrame(objs)
    
    return dataframe


def getSearchWords(dataframe,fieldtext):
    
    #Instanciando o objeto retirar palavras repetidas
    txt2matriObj = CountVectorizer()
    removestopwordsFull = []

    for rows in dataframe[fieldtext]:
        a = ''
        for row in rows:
            a += row+' '
        removestopwordsFull.append(a)
        
    txt2matri = txt2matriObj.fit_transform(removestopwordsFull)


    #Retornando as palavras sem repeticao
    return txt2matriObj.get_feature_names()
    

### Preparando as Expressões de busca

In [125]:
df = pd.read_csv(r'C:/Users/bernard-so/Documents/git/searchWords.csv',sep=';')

data = processCorpus(df,'text')
SearchWord = getSearchWords(data,'text')

#txt2matriObj = CountVectorizer()
#txt2matriObj.fit_transform(df)
#columns = txt2matriObj.get_feature_names()
#columns


['177',
 '247',
 '263',
 '270',
 '275',
 '283',
 '50',
 'abertura',
 'aceitabilidad',
 'alvará',
 'amostra',
 'anterior',
 'aptidão',
 'aquisição',
 'atestado',
 'avaliação',
 'bancária',
 'básico',
 'capacidad',
 'capit',
 'carta',
 'certidão',
 'cláusula',
 'competitividad',
 'comprovação',
 'condição',
 'conselho',
 'crea',
 'credenciamento',
 'critério',
 'deve',
 'débito',
 'econômico',
 'edit',
 'emolumento',
 'empresa',
 'especificação',
 'excessivo',
 'execução',
 'exigir',
 'exigência',
 'experiência',
 'fabricant',
 'falta',
 'financeira',
 'fiscal',
 'fiscalização',
 'fixo',
 'garantia',
 'global',
 'habilitação',
 'idoneidad',
 'ileg',
 'impugnaçõ',
 'inadequação',
 'infraçõ',
 'injustificada',
 'integralizado',
 'intervalo',
 'julgamento',
 'jurídica',
 'licitant',
 'licitação',
 'local',
 'marca',
 'município',
 'máximo',
 'mínimo',
 'negativa',
 'número',
 'objeto',
 'obra',
 'operacion',
 'parcelamento',
 'percentu',
 'permanent',
 'prazo',
 'presenci',
 'presencialment

In [104]:
folder = 'C:/Users/bernard-so/Documents/git/p/1992'
#files = glob.glob(os.path.join(folder,'*.txt'))
data = preProcessingCorpus(folder)
Newdf = processCorpus(data,'text')
Newdf = getCountWordsDocuments(Newdf,'text')
Newdf


,id,docid,text,year,ausência,critério,decisão,inverídica
0,0,AC-1-1992-1,"[tribun, conta, união, dado, materiai, decisão...",1992,0,0,3,0
0,1,AC-1-1992-2,"[tribun, conta, união, dado, materiai, decisão...",1992,2,1,7,1
0,2,AC-1-1992-P,"[tribun, conta, união, dado, materiai, decisão...",1992,0,0,3,0
0,3,AC-10-1992-1,"[tribun, conta, união, dado, materiai, decisão...",1992,0,0,3,0
0,4,AC-10-1992-2,"[tribun, conta, união, dado, materiai, decisão...",1992,0,0,6,0
0,5,AC-10-1992-P,"[tribun, conta, união, dado, materiai, decisão...",1992,0,6,11,0
0,6,AC-100-1992-1,"[tribun, conta, união, dado, materiai, decisão...",1992,0,0,5,0
0,7,AC-100-1992-2,"[tribun, conta, união, dado, materiai, decisão...",1992,0,0,4,0
0,8,AC-100-1992-P,"[tribun, conta, união, dado, materiai, decisão...",1992,0,0,5,0
0,9,AC-101-1992-1,"[tribun, conta, união, dado, materiai, decisão...",1992,0,0,4,0


In [8]:
stm = PorterStemmer()
listKeysWords = ['ausência', 'critério', 'decisão', 'inverídica']
suf = [stm.stem(txt) for txt in listKeysWords]
print(suf)

['ausência', 'critério', 'decisão', 'inverídica']


In [70]:
teste = Newdf.iloc[:3,:]
for index, row in teste.iterrows():
    print(index)

0
0
0


In [98]:
#rows, cols = Newdf.shape
#print(rows)
#objs = {'ausência':[0],'critério':[0],'decisão':[0],'inverídica':[0]}
#objs['ausência'] = [objs['ausência'][0] + 1]
#print(objs)
#objs['ausência'] = [objs['ausência'][0] + 1]
#word = 'ausência'
#if word in objs.keys():
#    print('Existe')
    
#print(list(objs.keys()))
listas = ['ausência', 'critério', 'decisão', 'inverídica']

it = iter(listas) 
zeros = [0]*len(listas)
print(dict(zip(it,zeros)))

#dict(listas)
#dict(zip(iter(listas),list(0)))

{'ausência': 0, 'critério': 0, 'decisão': 0, 'inverídica': 0}
